# Introduction

## Vision

Our goal for this project is to explore how machine learning can enhance the quality of academic essays and papers. We begin by selecting an existing essay and using a machine learning model to create an initial outline of it. This outline is then checked and corrected by a person to ensure it is accurate and relevant. Subsequently, we use the corrected outline to guide the ML model in writing a new version of the essay.

To evaluate the effectiveness of our method, we compare the original essay, written by a human, with the new one written by the model. We closely examine each word (token) in both essays and calculate how well each word fits. If the model's word is a better fit, we replace the original word with the AI's choice. This process allows us to experiment with different ways of providing information to the model, such as using just the original essay, the corrected outline, both together, the outline made by the model, or even no context at all.

Our aim is to determine which method of providing information enables the model to perform its best in terms of making fewer mistakes and maintaining clear and well-organized text. If our project is successful, it would demonstrate that AI can help improve the original essay, resulting in a final piece that is clear, well-written, and enhanced with the help of the model.

## Background

For this project, the primary data consists of essays from previous college courses written by the group members. This dataset was selected because it provides authentic academic writing samples that reflect the typical structures, themes, and complexity expected in college-level essays. These essays serve as an ideal basis for evaluating the effectiveness of machine learning models in generating and refining academic text, as they represent real-world examples of content that students are expected to produce. Our confidence in the primary dataset stems from several factors:
- Relevance: Essays from previous courses are directly relevant to the project's aim, which is to enhance the quality and coherence of academic essays using AI.
- Diversity: These essays encompass a range of subjects and writing styles, providing a robust base for testing the AI's adaptability and accuracy in generating and refining text.
- Authenticity: Using actual student essays ensures that the project results are applicable in real-world educational settings, enhancing the practical value of the research.

# Team & Responsibilities

- Tyler Arista
  - Responsibilties
- Lydia Powers
  - Responsbilities
- Chang Liu
  - Responsibilities

# Methodology

### Model Selection
We chose the GEMMA model from Google's Transformers library due to its advanced architecture that includes attention mechanisms, which are particularly effective for tasks involving natural language understanding and generation. The decision to utilize a pre-trained model and further fine-tune it on our specific dataset leverages the principle of transfer learning, which helps in achieving higher performance without the need for an extensive and computationally expensive training process from scratch.

### Data Preprocessing
Our dataset comprises academic essays, which we preprocessed by tokenizing using the tokenizer associated with our chosen model. This step is crucial as it converts raw text into a format that the model can understand, thereby reducing complexity and enhancing model training efficiency. Proper data structuring is key to minimizing semantic errors during generation.

### Training Methodology
The model was fine-tuned on our academic essay dataset using a specific batch size and learning rate. We monitored the training process for signs of overfitting by evaluating the loss on a validation set at each epoch. Adjustments to learning rates were made based on the model's performance on the validation set.

### Error Analysis
Error analysis involved examining the types of errors produced in the generated essays. This analysis is crucial for diagnosing issues with the model’s understanding and output, guiding subsequent rounds of fine-tuning.

### Performance Evaluation
We employed several metrics to evaluate the effectiveness of the model, including token loss and BLEU scores. These metrics help quantify how well the model performs in terms of generating coherent, grammatically correct, and contextually relevant text.


# Implementation<br>

## Introduction
Explain what this section will cover

## Setup and Configuration<br>
Detail the environment setup, including any libraries and frameworks used

In [244]:
!pip install textstat
!pip install bert-score
!pip install rouge
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [245]:
import torch, os
import textstat
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk import sent_tokenize, word_tokenize
from rouge import Rouge
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# Work around a bug in the version of PyTorch and GPU hardware curretnly on Kaggle. On other hardware, removing these lines may lead to a speed-up.
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Load the model
USE_INSTRUCTION_TUNED = True # we'll switch this to True partway through the lab
if USE_INSTRUCTION_TUNED:
    model_name = '/kaggle/input/gemma/transformers/1.1-2b-it/1'
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-1.1-2b-it"
else:
    model_name = "/kaggle/input/gemma/transformers/2b/2"
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.bfloat16)
streamer = TextStreamer(tokenizer)
# Silence a warning.
tokenizer.decode([tokenizer.eos_token_id]);

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Data Preparation

### P1's Essay Components
##### Prompt

In [126]:
p1_essay_prompt = '''In Homegoing, Pachinko, and Stories We Tell, the authors/filmmaker choose deliberate artistic strategies to present the histories they narrate. Discuss how the literary/filmic choices of Gyasi, Lee, and Polley are part of their overall theory about how the past should be constructed. In other words, how does the way in which they present their stories intersect with what they are trying to say in their stories? Are there commonalities in their aims? If so, what are they? Are there critical differences?'''

#### Outline given by the User

In [127]:
p1_outline_first_paragraph = '''
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.
'''

In [128]:
p1_outline_second_paragraph = '''
Main idea: Personal decisions directly shape characters' identities and futures in all three works, highlighting the authors' and filmmaker's focus on the impact of individual agency within broader historical and social contexts.
Explanation: This exploration of personal choice aligns with the creators' views that history is not merely a series of events but a complex tapestry woven from individual actions and their consequences.
Example from Pachinko: Sunja's pivotal decision to engage with Hansu, and its ramifications, showcase how personal mistakes and moral dilemmas are central to character development and plot progression.
Example from Homegoing: The character H’s experience with forced labor and subsequent physical development illustrates how personal endurance and adaptation to circumstances reflect broader historical forces like slavery and institutional racism.
Example from Stories We Tell: Harry’s narrative and his one-sided love affair demonstrate how personal perceptions can deeply influence one’s identity and the stories they choose to tell or believe.
'''

In [129]:
p1_outline_third_paragraph = '''
Main idea: Characters’ identities and life paths are significantly influenced by the actions and statuses of those around them and their ancestors, emphasizing the interconnectedness of personal histories within larger societal narratives.
Explanation: This theme underscores the authors' and filmmaker's perspective that individual lives are not isolated but are deeply affected by the historical and relational contexts in which they exist, echoing a broader theory that history is constructed collectively rather than singularly.
Example from Pachinko: Isak’s altruistic decision to marry Sunja provides a stark contrast to her initial dilemma, showing how benevolent actions from others can redirect an individual’s life trajectory dramatically.
Example from Homegoing: The legacy of slavery, as seen through Esi and her descendant Ness, highlights how ancestral histories cast long shadows over the lives of future generations, shaping identities and opportunities long after the original events have passed.
Example from Stories We Tell: The revelation of Sarah’s true paternity and Diane’s decisions regarding her upbringing illustrate the profound impact parental choices have on children’s identities and their understanding of family narratives.
'''

#### Human Written Essay

In [130]:
p1_essay_first_paragraph = '''Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. '''

p1_flesch_reading_ease_first_paragraph = textstat.flesch_reading_ease(p1_essay_first_paragraph)
print(f"Flesch Reading Ease: {flesch_reading_ease}")

Flesch Reading Ease: 69.82


In [131]:
p1_essay_second_paragraph = '''People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to marry her because he is already married. In her society, bearing a child without a father will lead to having the mother, in this case, Sunja, be disowned. It could also lead to her family and her child being disowned as well. Looking through the lens of Sunja, we could see how devastated she was and the impact her decisions could make on her in the future. Also, in the novel, Homegoing by Yaa Gyasi, we see through many different lenses in many different generations. In the chapter about H, it states, “The boss man was called Mr. John. He asked to take off his shirt. He inspected the muscles on his back and his arms and whistled. ‘Any man what can spend ten years working at Rock Slope and live to tell about it’s worth watching”(Gyasi 169). When H was sent to me while he was arrested, he might not have had an option on whether or not he wanted to work, but it made him better in the long run. He had gotten physically strong, and when he had gotten released, he was able to find a mining job that paid him. The situation he was put in shaped the person that H was. He became a hard worker and he made sure he did his job. Those were some of the things that he learned while he mined for the jail, but he was able to apply those lessons to the real world and shaped him to be somebody that was hirable. Another example of identities being defined by self choices is in the documentary, Stories We Tell, produced by Sarah Polley. When Harry met with Diane when she traveled to Montreal for a play, Harry fell in love with her. He had his own one-sided story, where he thought he was going to be with Diane for the rest of his life. This ultimately shaped his identity for the future. At the end of the documentary, Sarah asked Harry whether he liked that many people are sharing their sides of the story or if he disliked it. He shared that he did not like how many people shared their sides because he thought it was only his story to tell. He got wrapped up in the sense that Diane only loved him, and that changed the way he viewed his story.'''

p1_flesch_reading_ease_second_paragraph = textstat.flesch_reading_ease(p1_essay_second_paragraph)
print(f"Flesch Reading Ease: {flesch_reading_ease}")

Flesch Reading Ease: 69.82


In [132]:
p1_essay_third_paragraph = '''While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Sunja and her child’s back. They would no longer have to bear the weight of being dishonored by society and her family. Having people who can have a different perspective of the matter, can allow one to alter the course of somebody. Then, in the novel, Homegoing, we were able to witness the identities being altered because of their past relatives. In the novel, it states, “Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months”(Gyasi 70). Ness was a descendant of Esi in this story. Esi was a part of the slave trade system and her life was very rough. Esi went through an enormous amount of abuse and unbearable situations. Since being in that slave system, it had affected the rest of her family tree. Ness is the daughter of Esi, and so, Ness had to bear the identity of being a slave just like her mother. She had to be a slave on a plantation and no choice she made would be able to change that in this system. Her identity at the time was determined by her mother and how she was a part of the system. Lastly, in the documentary, Stories We Tell, Sarah’s identity was shaped by the choices her mother made when she was alive. Sarah never knew that Michael was not her biological father until she grew up. Sarah was able to form a strong bond with Michael, and that was because Diane decided to have Michael take care of her instead of Harry. Once Sarah found out that Harry was her father, Sarah was still able to keep the close relationship with Michael even after finding out the truth. Sarah was able to keep her identity from the one she formed with Michael, and that was all due to Diane.'''

p1_flesch_reading_ease_third_paragraph = textstat.flesch_reading_ease(p1_essay_third_paragraph)
print(f"Flesch Reading Ease: {flesch_reading_ease}")

Flesch Reading Ease: 69.82


### P2's Essay Components
#### Prompt

In [133]:
p2_prompt = '''Write a one-page essay based on the book Things Fall Apart by Chinua Achebe that discusses the relationship between Okonkwo and Ikemefuna'''

#### Outline given by the User

In [134]:
p2_outline_first_paragraph = '''
Brief introduction to the context of the scene where Okonkwo eats a locust.
Mention of Ezeudu’s arrival and the news he brings about Ikemefuna’s fate.'''

In [135]:
p2_outline_second_paragraph = '''
Description of Ezeudu as a respected elder and his unique position to confront Okonkwo.
Examination of Okonkwo’s motivations: fear of appearing weak and his disregard for Ezeudu’s advice.
'''

In [136]:
p2_outline_third_paragraph = '''
Reflection on Okonkwo’s fondness for Ikemefuna and the impact on his son Nwoye.
Analysis of the father-son dynamic between Okonkwo and Ikemefuna.
'''

In [137]:
p2_outline_forth_paragraph = '''
Exploration of Okonkwo’s internal conflict between his self-image and his love for Ikemefuna.
Detailed recount of the moment Okonkwo kills Ikemefuna, highlighting the betrayal and its emotional impact.
'''

In [138]:
p2_outline_fifth_paragraph = '''
Discussion of how Okonkwo’s perception of his own father influences his actions and self-perception.
Analysis of the significance of being called "father" by Ikemefuna and its implications on Okonkwo’s decision.
'''

In [139]:
p2_outline_sixth_paragraph = '''
Summation of how Okonkwo’s character complexities and his fear of weakness lead to tragic choices.
Reflection on the themes of identity and legacy in "Things Fall Apart" as illustrated through Okonkwo’s actions and decisions.
'''

#### Human Written Essay

In [140]:
p2_essay_first_paragraph = '''When I first start reading the passage, I think about what Okonkwo is eating. I have never eaten a locust and I am not sure if I would want to. But it is interesting that Okonkwo is eating something considered a rare delicacy when Ezeudu comes to tell him that Ikemefuna is going to die.'''

p2_flesch_reading_ease_first_paragraph = textstat.flesch_reading_ease(p2_essay_first_paragraph)
print(f"Flesch Reading Ease: {flesch_reading_ease}")

Flesch Reading Ease: 69.82


In [141]:
p2_essay_second_paragraph = '''The passage states that Ezeudu is the oldest man in the village and is given great respect. It makes me wonder if others felt the same way he did about Okonkwo and Ikemefuna’s relationship, but he is the only one in position to confront Okonkwo. I feel like Okonkwo acts too much in the present without thinking. He killed Ikemefuna because he did not want to be seen as weak. Ezeudu tells him not to but he does not worry about what Ezeudu will think of him, only what the people around him think.'''

p2_flesch_reading_ease_second_paragraph = textstat.flesch_reading_ease(p2_essay_second_paragraph)
print(f"Flesch Reading Ease: {p2_flesch_reading_ease_second_paragraph}")

Flesch Reading Ease: 77.77


In [142]:
p2_essay_third_paragraph = '''I cannot imagine caring about your own self image so much that you would be willing to kill someone you love. Not only that, Ikemefuna called to Okonkwo when the others attacked him. Ikemefuna trusted Okonkwo. But in the end Okonkwo ran to Ikemefuna and killed him instead of saving him. The hope that Ikemefuna must have felt when he saw his ‘father’ run toward him when he cried must have given way to terror when he saw Okonkwo raise his machete and strike him down.'''

p2_flesch_reading_ease_third_paragraph = textstat.flesch_reading_ease(p2_essay_third_paragraph)
print(f"Flesch Reading Ease: {p2_flesch_reading_ease_third_paragraph}")

Flesch Reading Ease: 79.4


In [143]:
p2_essay_forth_paragraph = '''I cannot imagine caring about your own self image so much that you would be willing to kill someone you love. Not only that, Ikemefuna called to Okonkwo when the others attacked him. Ikemefuna trusted Okonkwo. But in the end Okonkwo ran to Ikemefuna and killed him instead of saving him. The hope that Ikemefuna must have felt when he saw his ‘father’ run toward him when he cried must have given way to terror when he saw Okonkwo raise his machete and strike him down.'''

p2_flesch_reading_ease_forth_paragraph = textstat.flesch_reading_ease(p2_essay_forth_paragraph)
print(f"Flesch Reading Ease: {p2_flesch_reading_ease_forth_paragraph}")

Flesch Reading Ease: 79.4


In [144]:
p2_essay_fifth_paragraph = '''It makes me think of Okonkwo’s relationship with his father. How he hated his father for being lazy, and I wonder if his view of a father figure affected his decision as well. When Ezeudu talked to Okonkwo he said not to kill Ikemefuna because he called Okonkwo father. Maybe in his mind, although he cared for Ikemefuna, he saw himself in a bad light whenever he was called father.'''

p2_flesch_reading_ease_fifth_paragraph = textstat.flesch_reading_ease(p2_essay_fifth_paragraph)
print(f"Flesch Reading Ease: {p2_flesch_reading_ease_fifth_paragraph}")

Flesch Reading Ease: 70.63


In [145]:
p2_essay_sixth_paragraph = '''Reflecting on these moments in the story underscores the complexities of Okonkwo’s character and his conflicting roles as father, warrior, and man driven by the fear of appearing weak. This ultimately shapes his tragic choices, illuminating the broader themes of identity and legacy within the novel.'''

p2_flesch_reading_ease_sixth_paragraph = textstat.flesch_reading_ease(p2_essay_sixth_paragraph)
print(f"Flesch Reading Ease: {p2_flesch_reading_ease_sixth_paragraph}")

Flesch Reading Ease: 39.67


### P3's Essay Components
#### Prompt

In [146]:
p3_prompt = '''Evaluate the ethical implications of architectural design in public spaces, using Tiananmen Square as a case study.'''

#### Outline given by the User

In [147]:
p3_outline_first_paragraph = '''Overview of Tiananmen Square's significance and thesis statement.'''

In [148]:
p3_outline_second_paragraph = '''Explanation of Tiananmen Square's design to serve people and state leaders. Versatility for various events and ceremonies.'''

In [149]:
p3_outline_third_paragraph = '''Description of nearby crucial buildings and their roles. Analysis of how these buildings enhance Tiananmen Square's importance. Detailing the purposes of Tiananmen Square's three entrances. Illustration of the diverse events facilitated through each entrance.'''

In [150]:
p3_outline_forth_paragraph = '''Recap of Tiananmen Square's ethical architecture. Emphasis on its enduring significance in Chinese culture and history'''

#### Human Written Essay

In [151]:
p3_essay_first_paragraph = ''' Tiananmen Square is mostly ethical because it was designed to serve the people and the state leaders in many ways, it was surrounded by important buildings, and it had three entrances each serving in a different way. '''

p3_flesch_reading_ease_first_paragraph = textstat.flesch_reading_ease(p3_essay_first_paragraph)
print(f"Flesch Reading Ease: {p3_flesch_reading_ease_first_paragraph}")

Flesch Reading Ease: 42.38


In [152]:
p3_essay_second_paragraph = '''Tiananmen Square can be used as a place for reviewing troops, visiting, and all kinds of parades. In most independence days and army days, China will have a big reviewing troops ceremony going on in Tiananmen Square. Soldiers and all kinds of weaponed cars will march through Tiananmen Square. It shows to the world that China is a powerful country. Tiananmen Square is also a popular visit place for foreigners. There were millions of people visiting every year. It shows both the modern and ancient side of China. A lot of people might not know but Tiananmen Square also has a lot of parades going on every year. Especially during the spring festival, there will be lots of decorated cars and dancing people around Tiananmen Square to celebrate the coming of the new year. '''

p3_flesch_reading_ease_second_paragraph = textstat.flesch_reading_ease(p3_essay_second_paragraph)
print(f"Flesch Reading Ease: {p3_flesch_reading_ease_second_paragraph}")

Flesch Reading Ease: 73.27


In [153]:
p3_essay_third_paragraph = '''Tiananmen Square is located in the center of the city of Beijing which is also surrounded by lots of crucial buildings like the Chinese museum, the Great Hall of People, and the Forbidden City. On the east side of Tiananmen Square stands the Chinese museum. The Chinese museum is the largest comprehensive museum in China. It contains some of China’s most valuable national treasures. To the west is the Great Hall of People where the state leaders and the president hold all the important meetings. Lastly to the north was the Forbidden City. The Forbidden City is one of the biggest wooden structures in the world. It was built 600 years ago and the royal family of ancient China lived there for hundreds of years. Tiananmen Square was surrounded by all of those important architects. '''

p3_flesch_reading_ease_third_paragraph = textstat.flesch_reading_ease(p3_essay_third_paragraph)
print(f"Flesch Reading Ease: {p3_flesch_reading_ease_third_paragraph}")

Flesch Reading Ease: 64.71


In [154]:
p3_essay_forth_paragraph = '''It can be used in ways, it is surrounded by some of the most significant buildings in China, and even its three doors have different meanings. All of these reasons made Tiananmen Square an ethical building. '''

p3_flesch_reading_ease_forth_paragraph = textstat.flesch_reading_ease(p3_essay_forth_paragraph)
print(f"Flesch Reading Ease: {p3_flesch_reading_ease_forth_paragraph}")

Flesch Reading Ease: 70.13


## Token Loss Computation

The function below will figure out the loss for each word in an essay. It breaks the essay down into individual words and uses a model to predict what words might typically come next. For each word, it calculates how surprising that word is compared to what the model expects. This surprise factor, or loss, helps point out parts of the essay where the writing could be different from what's usually seen, showing where there might be room for improvement or unique style choices.

In [34]:
import torch
import pandas as pd

def analyze_text(essay):
    # Tokenize the essay for comparison
    essay_ids = tokenizer.encode(essay, return_tensors='pt').to(model.device)
    
    # Generate logits for the essay input
    with torch.no_grad():
        outputs = model(essay_ids)
        logits = outputs.logits
    
    # Analyze tokens for loss
    spans = []
    highest_loss = float('-inf')
    softmax = torch.nn.Softmax(dim=-1)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])
    
    for i in range(1, essay_ids.size(1)):  # Start from 1 to skip the first token (usually [CLS] or similar)
        probs = softmax(logits[0, i - 1])
        token_loss = -torch.log(probs[essay_ids[0, i]]).item()
        most_likely_token_id = torch.argmax(probs).item()
        token = essay_tokens[i]  # Adjust index for essay tokens
        most_likely_token = tokenizer.decode([most_likely_token_id])
        
        spans.append({
            'original_token': token,
            'token_loss': token_loss,
            'most_likely_token': most_likely_token,
            'loss_ratio': None  # To be calculated later
        })
        
        if token_loss > highest_loss:
            highest_loss = token_loss

    # Normalize loss ratios
    for span in spans:
        span['loss_ratio'] = span['token_loss'] / highest_loss if highest_loss != 0 else 0

    df = pd.DataFrame(spans)
    return df


### Token Loss for Each Original Human Written Paragraph(p1)

In [213]:
p1_analysis_df_w_no_context_first_paragraph = analyze_text(p1_essay_first_paragraph)
display(p1_analysis_df_w_no_context_first_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,Many,76.802307,increa,1.000000
1,▁people,1.230443,people,0.016021
2,▁believe,1.468063,believe,0.019115
3,▁that,0.070375,that,0.000916
4,▁there,4.952903,the,0.064489
...,...,...,...,...
125,▁including,15.139212,.,0.197119
126,▁their,1.828609,the,0.023809
127,▁ancestors,5.650682,families,0.073574
128,.,0.541190,.,0.007047


In [214]:
p1_analysis_df_w_no_context_second_paragraph = analyze_text(p1_essay_second_paragraph)

In [215]:
p1_analysis_df_w_no_context_third_paragraph = analyze_text(p1_essay_third_paragraph)

### Token Loss for Each Paragraph(P2)

In [116]:
p2_analysis_df_w_no_context_first_paragraph = analyze_text(p2_essay_first_paragraph)
display(p2_analysis_df_w_no_context_first_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,When,75.177307,increa,1.000000
1,▁I,4.480940,a,0.059605
2,▁first,4.088663,am,0.054387
3,▁start,1.920110,started,0.025541
4,▁reading,8.119610,a,0.108006
...,...,...,...,...
63,▁is,1.066990,has,0.014193
64,▁going,3.925479,dead,0.052216
65,▁to,0.012439,to,0.000165
66,▁die,2.679891,kill,0.035648


In [117]:
p2_analysis_df_w_no_context_second_paragraph = analyze_text(p2_essay_second_paragraph)

In [118]:
p2_analysis_df_w_no_context_third_paragraph = analyze_text(p2_essay_third_paragraph)

In [119]:
p2_analysis_df_w_no_context_forth_paragraph = analyze_text(p2_essay_forth_paragraph)

In [120]:
p2_analysis_df_w_no_context_fifth_paragraph = analyze_text(p2_essay_fifth_paragraph)

In [121]:
p2_analysis_df_w_no_context_sixth_paragraph = analyze_text(p2_essay_sixth_paragraph)

### Token Loss for Each Paragraph(P3)

In [40]:
p3_analysis_df_w_no_context_first_paragraph = analyze_text(p3_essay_first_paragraph)
display(p3_analysis_df_w_no_context_first_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,▁Tian,81.739807,increa,1.000000
1,an,6.269291,yi,0.076698
2,men,2.949823,",",0.036088
3,▁Square,4.097900,",",0.050133
4,▁is,1.951980,protests,0.023880
5,▁mostly,13.330235,a,0.163081
6,▁ethical,21.190880,known,0.259248
7,▁because,3.710687,?,0.045396
8,▁it,0.768544,it,0.009402
9,▁was,5.210018,is,0.063739


In [113]:
p3_analysis_df_w_no_context_second_paragraph = analyze_text(p3_essay_second_paragraph)

In [114]:
p3_analysis_df_w_no_context_third_paragraph = analyze_text(p3_essay_third_paragraph)

# Different Context Approaches To Create Paragraphs
## Generate an Outline from Human Written Essay & Revise that Outline to Generate a New Paragraph
In this section, we will use a ML model to convert a provided essay paragraph into a detailed outline. This outline will list the main ideas and supporting details. We then will use this outline to help the model write a new paragraph.

### P1's Essay
#### First Paragraph

In [175]:
%%time
doc = f'''Please break this introduction paragraph into a detailed outline format focusing on the thesis statement and a brief introduction: {p1_essay_first_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=300,
    do_sample=False,
    streamer=streamer
)

<bos>Please break this introduction paragraph into a detailed outline format focusing on the thesis statement and a brief introduction: Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors.  

**Thesis Statement:** This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narrat

In [182]:
doc = """
Using the outline and thesis provided below, compose a comprehensive introductory paragraph. The paragraph should seamlessly integrate the points from the outline into a cohesive narrative that sets the stage for the paper. This introduction should capture the essence of the thesis and outline, providing a clear and engaging overview of the topics to be discussed.

**Thesis Statement:** This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narratives of others can provide valuable insights into our own lives and the world around us.

**Detailed Outline for Introduction Paragraph:**

- Start by addressing the common belief of a singular narrative in the world, pointing out how prevalent this view is.
- Move on to highlight the underestimation of the multiplicity of stories, emphasizing the diversity and richness of different narratives.
- Introduce the authors and films discussed in the paper, such as Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.
- Explain how narratives shape our perspectives and understanding of the world, noting the influence of storytelling on personal and collective consciousness.
- Discuss the role of personal choices and the impact of others in shaping these narratives, connecting this discussion to the thesis.
- Highlight how the narratives of Homegoing, Pachinko, and Stories We Tell provide context and meaning, demonstrating the influence of personal agency and the choices made by individuals.
- Conclude by linking these discussions back to the thesis, reaffirming the importance of understanding diverse narratives for personal and collective growth, and setting the expectation for the rest of the paper.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=500,  # Adjusted token limit to allow for a more extensive development
    do_sample=False,  # Set to deterministic
    streamer=streamer
)

<bos>
Using the outline and thesis provided below, compose a comprehensive introductory paragraph. The paragraph should seamlessly integrate the points from the outline into a cohesive narrative that sets the stage for the paper. This introduction should capture the essence of the thesis and outline, providing a clear and engaging overview of the topics to be discussed.

**Thesis Statement:** This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narratives of others can provide valuable insights into our own lives and the world around us.

**Detailed Outline for Introduction Paragraph:**

- Start by addressing the common belief of a singular narrative in the world, pointing out how prevalent this view is.
- Move on to highlight the underestimation of the multiplicity of stories, emphasizing the diversity and richness of different narratives.
- Introduce the authors and films discussed in the 

In [183]:
p1_generated_first_paragraph_models_outline = '''
Despite the pervasive belief of a singular narrative dominating the human experience, the reality is far more nuanced. The multiplicity of stories that populate our world reflects the richness and complexity of human experience, offering a wealth of insights into our own lives and the world around us. This paper delves into the power of narratives, exploring how they shape our perspectives, influence personal and collective consciousness, and provide context and meaning. By examining the narratives of Yaa Gyasi, Min Jin Lee, and Sarah Polley, we will discover how personal choices and the choices of others shape the narratives we embrace, ultimately contributing to our personal growth and collective understanding.
'''
p1_analysis_df_w_models_outline_first_paragraph = analyze_text(p1_generated_first_paragraph_models_outline)
display(p1_analysis_df_w_models_outline_first_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,\n,73.802307
1,Despite,10.280128
2,▁the,0.391104
3,▁pervasive,8.788400
4,▁belief,5.043524
...,...,...
124,▁and,0.022032
125,▁collective,0.590187
126,▁understanding,0.084147
127,.,0.024807


#### Second Paragraph

In [47]:
%%time
doc = f'''Please convert the following detailed discussion into a structured outline, emphasizing the main thesis about how personal choices shape identities and providing key examples from the texts: {p1_essay_second_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>Please convert the following detailed discussion into a structured outline, emphasizing the main thesis about how personal choices shape identities and providing key examples from the texts: People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to mar

In [186]:
doc = """
Compose a detailed body paragraph based on the theme that personal choices shape identities.

**Main Idea:** Personal choices play a significant role in shaping identities by influencing the narratives and experiences individuals encounter throughout life.

**Detailed Outline for body Paragraph:**
- The impact of Sunja's decision with Hansu on her and her family's reputation.
- How H's time in prison made him a hardworking and responsible person.
- The effect of Harry's biased view on his own identity and story.
- Tie it back to how these stories illustrate the link between personal choices and societal expectations.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=500,  # Increase token limit to enhance detail
    do_sample=True,  # Enable sampling for more diverse responses
    streamer=streamer
)

<bos>
Compose a detailed body paragraph based on the theme that personal choices shape identities.

**Main Idea:** Personal choices play a significant role in shaping identities by influencing the narratives and experiences individuals encounter throughout life.

**Detailed Outline for body Paragraph:**
- The impact of Sunja's decision with Hansu on her and her family's reputation.
- How H's time in prison made him a hardworking and responsible person.
- The effect of Harry's biased view on his own identity and story.
- Tie it back to how these stories illustrate the link between personal choices and societal expectations.
- Discuss the transformative power of personal choice in overcoming adversity, achieving dreams, and fostering personal growth.

**Body Paragraph:**

Personal choices act as pivotal elements in shaping identities, influencing the narratives and experiences individuals encounter throughout life. The instances of Sunja and Hansu illustrate how external events and perso

In [187]:
p1_generated_second_paragraph_models_outline = '''
Personal choices act as pivotal elements in shaping identities, influencing the narratives and experiences individuals encounter throughout life. The instances of Sunja and Hansu illustrate how external events and personal choices intertwined, shaping their narratives and reputations. Hansu's time in prison transformed him into a hardworking and responsible individual, demonstrating the power of personal accountability and resilience. Conversely, Harry's biased view of the world impacted not only his perception of himself but also shaped his course of actions. These stories highlight the intricate link between personal choices and societal expectations. Individuals are not merely the products of their environments but actively create their identities through their choices, shaping their self-perception, values, and aspirations. Personal agency empowers individuals to navigate challenging situations, pursue their dreams, and cultivate personal growth.
By exploring the choices of these characters, we gain insights into the transformative power of personal choice in overcoming adversity, achieving personal fulfillment, and fostering personal growth. It is evident that personal choices have the potential to shape not only individual identities but also the broader narratives of communities and societies.
'''
p1_analysis_df_w_models_outline_second_paragraph = analyze_text(p1_generated_second_paragraph_models_outline)
display(p1_analysis_df_w_models_outline_second_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,\n,73.802307
1,Personal,14.702003
2,▁choices,12.514207
3,▁act,11.764960
4,▁as,0.032671
...,...,...
207,▁communities,0.441326
208,▁and,0.108655
209,▁societies,0.047245
210,.,0.157159


#### Third Paragraph

In [188]:
%%time
doc = f'''Please transform the following detailed discussion into a structured outline, emphasizing the thesis on how external influences like family and societal contexts shape individual identities, and include key examples from the texts: {p1_essay_third_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=300,
    do_sample=False,
    streamer=streamer
)

<bos>Please transform the following detailed discussion into a structured outline, emphasizing the thesis on how external influences like family and societal contexts shape individual identities, and include key examples from the texts: While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Su

In [198]:
doc = """
Compose a detailed body paragraph on how external influences shape individual identities, particularly through family and societal contexts. Use direct quotes from 'Pachinko', 'Homegoing', and 'Stories We Tell' to support your analysis.

- From 'Pachinko', discuss how Yangjin's decision reflects societal pressures: "‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'" (Lee 74).
- From 'Homegoing', analyze Ness's identity shaped by her forced labor conditions: "Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months" (Gyasi 70).
- From 'Stories We Tell', reflect on how Sarah's understanding of her identity changes after learning about her paternity: "Sarah’s identity was shaped by the choices her mother made when she was alive. Sarah never knew that Michael was not her biological father until she grew up" (Documentary).

Conclude by tying these examples back to the broader impact of external influences on personal identity formation, illustrating how these forces dictate individual choices and life paths.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=800,  # Extended limit for comprehensive development
    do_sample=False,  # Disable sampling to focus on generating more accurate responses
    streamer=streamer
)

<bos>
Compose a detailed body paragraph on how external influences shape individual identities, particularly through family and societal contexts. Use direct quotes from 'Pachinko', 'Homegoing', and 'Stories We Tell' to support your analysis.

- From 'Pachinko', discuss how Yangjin's decision reflects societal pressures: "‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'" (Lee 74).
- From 'Homegoing', analyze Ness's identity shaped by her forced labor conditions: "Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months" (Gyasi 70).
- From 'Stories We Tell', reflect on how Sarah's understanding of her identity changes after learning about her paternity: "Sarah’s identity was shaped by the choices her mother made when she was alive. Sarah never knew that Mi

In [199]:
p1_generated_third_paragraph_models_outline = '''
External influences play a significant role in shaping individual identities, particularly through the contexts of family and society. As illustrated through the narratives of 'Pachinko', 'Homegoing', and 'Stories We Tell', individuals are shaped by societal pressures, forced labor conditions, and the revelation of unexpected personal information.
In 'Pachinko', Yangjin's decision reflects societal pressures, where a daughter's life is valued more for her potential than her own happiness. Similarly, Ness's identity is shaped by the brutal realities of forced labor, where physical and emotional suffering become defining aspects of her being.
The revelation of Sarah's paternity in 'Stories We Tell' highlights the unexpected nature of identity formation. It forces her to confront the complexities of family relationships and the challenges of piecing together her own personal history. These examples illustrate how external influences can disrupt existing identities and necessitate the formation of new ones.
In conclusion, external influences shape individual identities by dictating choices and life paths. From societal pressures and forced labor to unexpected personal information, these forces influence how individuals perceive themselves and navigate their place in the world. Understanding these external influences is crucial for comprehending the diversity of human experiences and the complexities of personal identity formation.
'''
p1_analysis_df_w_models_outline_third_paragraph = analyze_text(p1_generated_third_paragraph_models_outline)
display(p1_analysis_df_w_models_outline_third_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,\n,73.802307
1,External,16.577003
2,▁influences,7.993199
3,▁play,5.645504
4,▁a,0.006582
...,...,...
245,▁personal,1.748683
246,▁identity,0.225315
247,▁formation,0.088666
248,.,0.000518


### p2's Essay

### P3's Essay

## Model Generates Paragraph from Human Written Outline
This approach uses a ready-made outline created by a human to help write a detailed paragraph. Unlike the other sections, where maybe the model makes its own outline from an essay and then write a paragrpah, here the outline is already set. This method will show how we can use a clear plan to directly create a well-organized paragraph that discusses specific ideas the writer wants.

### P1's Essay
#### First Paragraph

In [53]:
doc = f"""
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.{p1_outline_first_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.

This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal an

In [168]:
p1_generated_first_paragraph_human_outline = '''This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal and ancestral choices are not merely individual experiences but are also shaped by broader historical contexts, familial dynamics, and societal narratives.This paper seeks to demonstrate how authors and filmmakers employ multiple perspectives to illuminate the multifaceted nature of human experience, emphasizing the role of personal narratives in understanding the complexities of identity and history.'''
p1_analysis_df_w_human_outline_first_paragraph = analyze_text(p1_generated_first_paragraph_human_outline)
display(p1_analysis_df_w_human_outline_first_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,This,72.989807
1,▁paper,10.340496
2,▁investigates,2.858632
3,▁how,4.138315
4,▁authors,12.082332
...,...,...
115,▁of,0.022485
116,▁identity,0.638408
117,▁and,0.243705
118,▁history,0.112638


#### Second Paragraph

In [169]:
doc = f"""
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

Outline:
{p1_outline_second_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=300,
    do_sample=False,
    streamer=streamer
)


<bos>
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

Outline:

Main idea: Personal decisions directly shape characters' identities and futures in all three works, highlighting the authors' and filmmaker's focus on the impact of individual agency within broader historical and social contexts.
Explanation: This exploration of personal choice aligns with the creators' views that history is not merely a series of events but a complex tapestry woven from individual actions and their consequences.
Example from Pachinko: Sunja's pivotal decision to engage with Hansu, and its ramifications, showcase how personal mistakes and moral dilemmas are central to character development and plot progression.
Example from Homegoing: The character H’s experi

In [170]:
p1_generated_second_paragraph_human_outline = '''By examining these works through a lens of personal agency, this analysis underscores the importance of individual choice in shaping narratives and understanding the complex interplay between personal experiences, historical contexts, and societal structures. The narratives of Sunja, H, and Harry in Pachinko, Homegoing, and Stories We Tell reveal a recurring theme of the profound impact of personal decisions on shaping characters' identities and futures. Authors and filmmakers alike emphasize the significance of individual agency within broader historical and social contexts. This exploration aligns with the creators' belief that history is not merely a series of events but a tapestry woven from individual actions and their consequences. Through close examination of these works, we witness how personal choices serve as catalysts for character development and plot progression. Sunja's decision to engage with Hansu, despite societal disapproval, exemplifies how individual agency can lead to unexpected consequences and personal growth. H's experience with forced labor compels him to confront the limitations of his own body and the systemic oppression he has endured. Harry's narrative, marked by his one-sided love affair, highlights how personal perceptions can shape one's identity and the stories they choose to tell.'''
p1_analysis_df_w_human_outline_second_paragraph = analyze_text(p1_generated_second_paragraph_human_outline)
display(p1_analysis_df_w_human_outline_second_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,By,76.927307
1,▁examining,5.146006
2,▁these,5.455596
3,▁works,7.330101
4,▁through,5.878495
...,...,...
225,▁they,2.904954
226,▁choose,0.116842
227,▁to,0.000144
228,▁tell,0.014521


#### Third Paragraph

In [172]:
doc = f"""
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

Outline:
{p1_outline_third_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=350,
    do_sample=False,
    streamer=streamer
)


<bos>
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

Outline:

Main idea: Characters’ identities and life paths are significantly influenced by the actions and statuses of those around them and their ancestors, emphasizing the interconnectedness of personal histories within larger societal narratives.
Explanation: This theme underscores the authors' and filmmaker's perspective that individual lives are not isolated but are deeply affected by the historical and relational contexts in which they exist, echoing a broader theory that history is constructed collectively rather than singularly.
Example from Pachinko: Isak’s altruistic decision to marry Sunja provides a stark contrast to her initial dilemma, showing how benevolent actions from 

In [173]:
p1_generated_third_paragraph_human_outline = '''By exploring the interconnectedness of personal histories, this theme underscores the power of social context in shaping individual lives and emphasizes the importance of acknowledging the collective and historical forces that influence personal experiences. The intricate tapestry of human identity is intricately woven with the threads of those around us and our ancestors, creating a complex and interconnected narrative that unfolds through generations. This interconnectedness emerges from the shared experiences, histories, and social contexts that shape the lives of individuals and communities. As illustrated by the narratives of Isak, Esi, and Diane, individuals are not isolated entities but are shaped by the benevolent actions of others, the legacy of the past, and the choices made by their parents and ancestors. This interconnectedness is evident in the choices and decisions made by characters in literary works. Isak’s decision to marry Sunja highlights the transformative power of benevolent actions, redirecting her life trajectory towards a more fulfilling path. Esi and her descendant Ness bear the burden of the legacy of slavery, shaping their identities and opportunities long after the original events have passed. Similarly, Diane’s decision to raise her children based on her own experiences underscores the profound impact of parental choices on children’s identities and their understanding of family narratives. By exploring the interconnectedness of personal histories, this theme underscores the power of social context in shaping individual lives. It challenges the notion of individual agency and emphasizes the role of social forces in determining personal experiences. This understanding is crucial for fostering empathy and promoting social justice, as it encourages us to acknowledge the interconnectedness of our world and to work towards creating a more just and equitable society.'''
p1_analysis_df_w_human_outline_third_paragraph = analyze_text(p1_generated_third_paragraph_human_outline)
display(p1_analysis_df_w_human_outline_third_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,By,76.927307
1,▁exploring,8.208507
2,▁the,0.343502
3,▁interconnected,5.428399
4,ness,0.016759
...,...,...
317,▁just,0.586601
318,▁and,0.000334
319,▁equitable,0.006663
320,▁society,0.284619


### P2's Essay

### P3's Essay

## Generated Introduction Paragraph given Human Written Essay & Human Written Outline
In this approach, we use a detailed outline and complete essay both created by a human to guide the model in writing a new essay(split into paragraphs). This is different from other sectiosn because the outlien and essay are already given as context, and the model combines them to write a paragraph that fits well with the outlined points and the essay's content. This approach should demonstrate how machine learning can effectively use provided materials to make new, well-formed paragraphs that match the original essay's tone and structure

### p1's Essay
#### First Paragraph

In [257]:
doc = f"""
Generate an introduction paragraph leveraging the provided essay context and outline to craft a coherent and engaging narrative. The introduction should seamlessly integrate the themes outlined in the essay context and the structure provided in the outline, offering a compelling overview of the essay's argument and significance.

**Essay Context:** {p1_essay_first_paragraph}

**Outline for Introduction:**
{p1_outline_first_paragraph}

"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=500,  # Adjusted for potentially longer introductory needs
    do_sample=False,  # Disabled to ensure focus on the task
    streamer=streamer
)

<bos>
Generate an introduction paragraph leveraging the provided essay context and outline to craft a coherent and engaging narrative. The introduction should seamlessly integrate the themes outlined in the essay context and the structure provided in the outline, offering a compelling overview of the essay's argument and significance.

**Essay Context:** Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the peopl

In [262]:
p1_generated_first_paragraph_human_outline_and_essay = '''In the tapestry of human existence, where narratives intertwine, revealing the complexities of our individual journeys, we often find ourselves questioning the singular, dominant narrative that pervades our understanding of the world. While the individual may hold significant power in shaping their personal narrative, the broader context of history and the choices of those who came before them also play a crucial role in defining our identities and destinies. This essay explores how authors and filmmakers utilize multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, challenging our assumptions about the nature of history and storytelling.'''
p1_analysis_df_w_human_outline_and_essay_first_paragraph = analyze_text(p1_generated_first_paragraph_human_outline_and_essay)
display(p1_analysis_df_w_human_outline_and_essay_first_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,In,72.739807
1,▁the,0.542588
2,▁tapestry,8.662266
3,▁of,0.000673
4,▁human,2.316590
...,...,...
107,▁of,0.007170
108,▁history,5.449386
109,▁and,0.034599
110,▁storytelling,4.338239


#### Second Paragraph

In [263]:
doc = f"""
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

**Essay Context:** {p1_essay_second_paragraph}

**Outline for Second Paragraph:**
{p1_outline_second_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=500,  # Adjusted for potentially longer paragraph needs
    do_sample=False,  # Disabled to ensure focus on the task
    streamer=streamer
)

<bos>
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

**Essay Context:** People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made 

In [266]:
p1_generated_second_paragraph_human_outline_and_essay = '''In conclusion, the exploration of personal choice in these three works underscores the transformative power of individual agency within broader historical and social contexts. By examining how characters navigate their choices and their consequences, these narratives challenge the notion of predetermined destinies and emphasize the importance of personal responsibility and agency in shaping one’s identity and future.'''
p1_analysis_df_w_human_outline_and_essay_second_paragraph = analyze_text(p1_generated_second_paragraph_human_outline_and_essay)
display(p1_analysis_df_w_human_outline_and_essay_second_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,In,7.273981e+01
1,▁conclusion,1.360509e+01
2,",",3.404864e-03
3,▁the,7.178910e-01
4,▁exploration,5.447742e+00
...,...,...
58,s,8.344654e-07
59,▁identity,3.248956e+00
60,▁and,1.116658e-01
61,▁future,1.100144e+00


#### Third Paragraph

In [265]:
doc = f"""
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

**Essay Context:** {p1_essay_third_paragraph}

**Outline for Second Paragraph:**
{p1_outline_third_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=600,  # Adjusted for potentially longer paragraph needs
    do_sample=False,  # Disabled to ensure focus on the task
    streamer=streamer
)

<bos>
Generate a detailed academic body paragraph that explores and expands upon the themes outlined below. The paragraph should delve into the details, providing a thorough analysis and discussion of the specific aspects of the topic mentioned in the outline and illustrated by the referenced paragraph.

**Essay Context:** While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the id

In [267]:
p1_generated_third_paragraph_human_outline_and_essay = '''By exploring how characters’ identities are shaped by the actions and contexts of others, this theme underscores the interconnectedness of personal histories and the role of history in shaping individual lives.'''
p1_analysis_df_w_human_outline_and_essay_third_paragraph = analyze_text(p1_generated_third_paragraph_human_outline_and_essay)
display(p1_analysis_df_w_human_outline_and_essay_third_paragraph[['original_token', 'token_loss']])

,original_token,token_loss
0,By,76.927307
1,▁exploring,8.208507
2,▁how,4.609127
3,▁characters,7.782189
4,’,5.707983
5,▁identities,5.345525
6,▁are,1.201509
7,▁shaped,0.231460
8,▁by,0.643914
9,▁the,2.139013


### P2's Essay

### P3's Essay

## Compare the Two Options (Human Written vs ML Model Written) & Highlight Where Changes are Made
This function helps visualize changes made to an essay. It first splits the essay into individual words. Them, using a list of suggested changes, it checks if any of the original words were recommended to be changed. If a word was changed, the function highlights the new word by wrapping it in brackets. Finally, it reassembles the essay, now with the highlighted changes, making it easy to see where edits have been made

In [43]:
def highlight_revised_essay(essay, analysis_df):
    tokens = essay.split()  # Split the original essay into tokens (simple space-based tokenization might not perfectly match tokenizer behavior)
    revised_tokens = tokens.copy()  # Create a copy of the tokens for modification
    
    token_idx_to_replace = {}  # Dictionary to map original token index to most likely token
    
    # Iterate over the dataframe to replace tokens
    for index, row in analysis_df.iterrows():
        original_token = row['original_token']
        most_likely_token = row['most_likely_token']
        token_loss = row['token_loss']
        
        # Find the index of the token in the original tokens list
        # This assumes exact match; handling multiple occurrences or mismatches might require more sophisticated tracking
        try:
            token_index = tokens.index(original_token)
            if tokens[token_index] != most_likely_token:  # Check if the token is actually different
                token_idx_to_replace[token_index] = most_likely_token
        except ValueError:
            # If the token is not found due to differences in tokenization, ignore it
            continue
    
    # Replace and mark tokens in the copied list
    for idx, replacement in token_idx_to_replace.items():
        revised_tokens[idx] = f"[{replacement}]"  # Wrap the replacement in brackets to highlight it
    
    # Join the revised tokens back into a single string
    revised_essay = ' '.join(revised_tokens)
    return revised_essay

# Essay Results(WRITTEN TEXT)

# Original Essay for Reference

In [222]:
print("Original Essay(P1):\n\n", p1_essay_first_paragraph + "\n\n" + p1_essay_second_paragraph + "\n\n" + p1_essay_third_paragraph)

Original Essay(P1):

 Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 

People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee

## Generated Introduction Paragraph Based On Most_likely_token(NO CONTEXT, just most_likely_token Chosen)

In [235]:
p1_most_likely_token_first_paragraph = "".join(p1_analysis_df_w_no_context_first_paragraph['most_likely_token'])
p1_most_likely_token_second_paragraph = "".join(p1_analysis_df_w_no_context_second_paragraph['most_likely_token'])
p1_most_likely_token_third_paragraph = "".join(p1_analysis_df_w_no_context_third_paragraph['most_likely_token'])

print("Revised Essay Based on Most Likely Tokenn/No Context(P1):\n", p1_most_likely_token_first_paragraph + "\n" + p1_most_likely_token_second_paragraph + "\n" + p1_most_likely_token_third_paragraph)

Revised Essay Based on Most Likely Tokenn/No Context(P1):
  increa people believe that the is a one true to on in the universe. and that call unwilling ones characters in This believe that their that fit around them and and that else is the story is simply supporting character character.

 need see't see about there are multiple of other stories happening at at once same time, but is make lead our perception. the focus and

 idea of artists who these Alone explore Chiinko, and Things of Tell use for their concept of African characters to the situations to explore the the power are these ofs lives intertw intertw interconnected by influenced by the choices they make. and and how impact they them. the families.


 increa oftens Republic are shaped shaped by their roles they make in life past. and consciously were good or unsuccessful. and or dishonorable.

 choices of editors who this narratives often a choices person’s identity define shape their destiny and

 this context “ “inko, Y Ji

## Generate an Outline from Human Written Essay & Revise that Outline to Generate a New Paragraph
### P1's Essay

In [210]:
print("\nEssay Generated by Model:\n", p1_generated_first_paragraph_models_outline + "\n" + p1_generated_second_paragraph_models_outline + "\n" + p1_generated_third_paragraph_models_outline)


Essay Generated by Model:
 
Despite the pervasive belief of a singular narrative dominating the human experience, the reality is far more nuanced. The multiplicity of stories that populate our world reflects the richness and complexity of human experience, offering a wealth of insights into our own lives and the world around us. This paper delves into the power of narratives, exploring how they shape our perspectives, influence personal and collective consciousness, and provide context and meaning. By examining the narratives of Yaa Gyasi, Min Jin Lee, and Sarah Polley, we will discover how personal choices and the choices of others shape the narratives we embrace, ultimately contributing to our personal growth and collective understanding.


Personal choices act as pivotal elements in shaping identities, influencing the narratives and experiences individuals encounter throughout life. The instances of Sunja and Hansu illustrate how external events and personal choices intertwined, sh

## Generated Introduction Paragraph Given Desired Human Outline

In [208]:
print("\nEssay Generated by Model:\n\n", p1_generated_first_paragraph_human_outline + "\n\n" + p1_generated_second_paragraph_human_outline + "\n\n" + p1_generated_third_paragraph_human_outline)


Essay Generated by Model:

 This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal and ancestral choices are not merely individual experiences but are also shaped by broader historical contexts, familial dynamics, and societal narratives.This paper seeks to demonstrate how authors and filmmakers employ multiple perspectives to illuminate the multifaceted nature of human experience, emphasizing the role of personal narratives in understanding the complexities of identity and history.

By examining these works through a lens of personal agency, this analysis underscores the importance of individual choice in shaping narratives and understanding the complex interplay between personal expe

## Genereated Introduction Paragraph Given Human Essay & Outline

In [268]:
print("\nEssay Generated by Model:\n\n", p1_generated_first_paragraph_human_outline_and_essay + "\n\n" + p1_generated_second_paragraph_human_outline_and_essay + "\n\n" + p1_generated_third_paragraph_human_outline_and_essay)


Essay Generated by Model:

 In the tapestry of human existence, where narratives intertwine, revealing the complexities of our individual journeys, we often find ourselves questioning the singular, dominant narrative that pervades our understanding of the world. While the individual may hold significant power in shaping their personal narrative, the broader context of history and the choices of those who came before them also play a crucial role in defining our identities and destinies. This essay explores how authors and filmmakers utilize multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, challenging our assumptions about the nature of history and storytelling.

In conclusion, the exploration of personal choice in these three works underscores the transformative power of individual agency within broader historical and social contexts. By examining how characters navigate their choices and their consequences, these narra

# Results
Include quantitative(tables, plots) and qualitiative(examples) results, including comparisons with similar work if applicable.

## BLEU SCORES
This score will suggest how closely the generated text matches the original essay, with higher scores indicating better similarity

In [269]:
def calculate_bleu(original_text, generated_text):
    smoothie = SmoothingFunction().method4
    original_sentences = sent_tokenize(original_text)
    generated_sentences = sent_tokenize(generated_text)
    
    bleu_scores = [sentence_bleu([word_tokenize(orig)], word_tokenize(gen), smoothing_function=smoothie) for orig, gen in zip(original_sentences, generated_sentences)]
    
    average_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    return average_bleu

In [270]:
original_essay = p1_essay_first_paragraph + "\n\n" + p1_essay_second_paragraph + "\n\n" + p1_essay_third_paragraph

# Generate texts using different contexts (Example usage, replace with actual generated texts)
no_context_text = p1_most_likely_token_first_paragraph + "\n" + p1_most_likely_token_second_paragraph + "\n" + p1_most_likely_token_third_paragraph
model_outline_text = p1_generated_first_paragraph_models_outline + "\n" + p1_generated_second_paragraph_models_outline + "\n" + p1_generated_third_paragraph_models_outline
human_outline_text = p1_generated_first_paragraph_human_outline + "\n" + p1_generated_second_paragraph_human_outline + "\n" + p1_generated_third_paragraph_human_outline
model_and_human_outline_text = p1_generated_first_paragraph_human_outline_and_essay + "\n" + p1_generated_second_paragraph_human_outline_and_essay + "\n" + p1_generated_third_paragraph_human_outline_and_essay

# Compute BLEU scores for each generation method
bleu_scores = {
    'No Context': calculate_bleu(original_essay, no_context_text),
    'Model Outline': calculate_bleu(original_essay, model_outline_text),
    'Human Outline': calculate_bleu(original_essay, human_outline_text),
    'Model and Human Outline': calculate_bleu(original_essay, model_and_human_outline_text)

}

# Print BLEU scores
for context, score in bleu_scores.items():
    print(f"BLEU Score for {context}: {score:.4f}")

BLEU Score for No Context: 0.1311
BLEU Score for Model Outline: 0.1902
BLEU Score for Human Outline: 0.1860
BLEU Score for Model and Human Outline: 0.1930


## ROGUE SCORES
This score evaluates the quality of the generated text by measuring its overlap with the original essay in terms of meaningful content.

In [271]:
def calculate_rouge_scores(reference, generated):
    rouge = Rouge()
    scores = rouge.get_scores(generated, reference)
    return scores

In [272]:
rouge_scores_no_context = calculate_rouge_scores(original_essay, no_context_text)
rouge_scores_model_outline = calculate_rouge_scores(original_essay, model_outline_text)
rouge_scores_human_outline = calculate_rouge_scores(original_essay, human_outline_text)
rouge_scores_model_and_human_outline = calculate_rouge_scores(original_essay, model_and_human_outline_text)

print("ROUGE Scores for No Context:\n", rouge_scores_no_context)
print("\nROUGE Scores for Model Outline:\n", rouge_scores_model_outline)
print("\nROUGE Scores for Human Outline:\n", rouge_scores_human_outline)
print("\nROUGE Scores for Model and Human Outline:\n", rouge_scores_model_and_human_outline)

ROUGE Scores for No Context:
 [{'rouge-1': {'r': 0.4431818181818182, 'p': 0.44419134396355353, 'f': 0.44368600182594503}, 'rouge-2': {'r': 0.16363636363636364, 'p': 0.15644171779141106, 'f': 0.1599581758702734}, 'rouge-l': {'r': 0.4159090909090909, 'p': 0.4168564920273349, 'f': 0.41638224755973346}}]

ROUGE Scores for Model Outline:
 [{'rouge-1': {'r': 0.14545454545454545, 'p': 0.256, 'f': 0.18550724175593375}, 'rouge-2': {'r': 0.026737967914438502, 'p': 0.057208237986270026, 'f': 0.03644314434679692}, 'rouge-l': {'r': 0.1340909090909091, 'p': 0.236, 'f': 0.17101448813274536}}]

ROUGE Scores for Human Outline:
 [{'rouge-1': {'r': 0.225, 'p': 0.36666666666666664, 'f': 0.2788732347232692}, 'rouge-2': {'r': 0.04385026737967915, 'p': 0.08488612836438923, 'f': 0.05782792216530002}, 'rouge-l': {'r': 0.21363636363636362, 'p': 0.34814814814814815, 'f': 0.26478872768101575}}]

ROUGE Scores for Model and Human Outline:
 [{'rouge-1': {'r': 0.09318181818181819, 'p': 0.36607142857142855, 'f': 0.148

## FLESH READING EASE SCORES
This score provides insight into the readability of the generated text, with higher scores indicating easier comprehension for readers.

In [273]:
def calculate_flesch_reading(text):
    score = textstat.flesch_reading_ease(text)
    return score

In [274]:
flesch_score_no_context = calculate_flesch_reading(no_context_text)
flesch_score_model_outline = calculate_flesch_reading(model_outline_text)
flesch_score_human_outline = calculate_flesch_reading(human_outline_text)
flesch_score_model_and_human_outline = calculate_flesch_reading(model_and_human_outline_text)

print("Flesch Reading Ease Score for No Context:", flesch_score_no_context)
print("Flesch Reading Ease Score for Model Outline:", flesch_score_model_outline)
print("Flesch Reading Ease Score for Human Outline:", flesch_score_human_outline)
print("Flesch Reading Ease Score for Model and Human Outline:", flesch_score_model_and_human_outline)

Flesch Reading Ease Score for No Context: 68.5
Flesch Reading Ease Score for Model Outline: 16.02
Flesch Reading Ease Score for Human Outline: 19.4
Flesch Reading Ease Score for Model and Human Outline: 14.12


# Implications
Discuss the social and ethical implications of using the technologies you've chosen for your project